# Test POCS

You can use this notebook to run some simple tests on POCS and some of the attached hardware.

Use **Shift-Enter** to run each cell and advance to the next one.  You can use **Ctrl-Enter** to run the cell without advancing.

See the [Official JupyterLab User Guider](https://jupyterlab.readthedocs.io/en/latest/user/index.html) for a comprehensive guide to JupyterLab.

## Loading POCS

Below we load some helper functions that will attach to our hardware and create a `POCS` object.

The `POCS` object is responsible for controlling the `Observatory` object. Think of `POCS` as the brains of your PANOPTES unit.

An `Observatory` contains details about your unit, such as connections to all the hardware, information about where it is physically located and what time it is, a scheduler that can find targets to look at, and more!

When your unit runs on its own each night all of these details are taken care of for you by the `POCS` object. In this notebook we will be the brains controlling the `Observatory` so we can test things.

In [ ]:
from panoptes.pocs.core import POCS
from panoptes.pocs.observatory import Observatory
from panoptes.pocs.mount import create_mount_from_config
from panoptes.pocs.camera import create_cameras_from_config
from panoptes.pocs.scheduler import create_scheduler_from_config
from panoptes.utils.time import current_time


def create_pocs(simulators=None):
    scheduler = create_scheduler_from_config()
    mount = create_mount_from_config()
    cameras = create_cameras_from_config()

    observatory = Observatory(cameras=cameras, mount=mount, scheduler=scheduler)

    # Add simulators if necessary before running.
    pocs = POCS(observatory, simulators=simulators or list())
    return pocs


After loading the code into the notebook above, we next want to create an individual `POCS` object (called an "instance" of `POCS`), which we assign to the `pocs` variable.

Note that you will see some messages being output during some of the commands.

In [ ]:
pocs = create_pocs()

After we have created the `pocs` instance, we need to initialize it. This is where all the connections to the hardware are created, so if something is hooked up incorrectly you will likely get errors here.

In [ ]:
pocs.initialize()

# Mount testing commands

Now that we have initialized `pocs`, we want to test that our mount works as expected. To save ourselves some typing, we make a new variable (`mount`) that points to the mount instance in `pocs`.

> **Warning** 
> 
> Before testing the mount, make sure that you have balanced the mount and that there are no cables or other obstructions that may interfere with the mount movement. See the [Balancing](https://www.projectpanoptes.org/build/hardware/mount-and-pier/mount/balancing-the-mount) page for more info.
> 
> Watch the mount carefully and be ready to physically turn off the power to the mount if it looks like something is going wrong.

In [ ]:
# We don't want to type this out every time.
mount = pocs.observatory.mount

The `mount` can be given a number of commands. The most useful for testing will be to have the mount automatically find the "home" (or "zero") position using the `search_for_home` command. Because we are manually controlling the mount we first need to unpark it before we make any movement commands.

In [ ]:
mount.unpark()

In [ ]:
mount.search_for_home()

You can also use `move_direction` to mimic pushing the arrow buttons on the hand controller.

In [ ]:
mount.move_direction(direction='east', seconds=1)
mount.move_direction(direction='south', seconds=3)

Finally, you can `park` the mount to test that your unit knows how to sleep at the end of a long night! 

The `park` command will take a few seconds to run and will first return to home before moving to the park position. Parking will consist of two movements, first around the RA axis and then around the Declination. If you have the camera box connected the lenses should be facing down after parking, otherwise see the section on [Saddle Plate Rotation](https://www.projectpanoptes.org/build/hardware/mount-and-pier/mount/saddle-plate-rotation).

In [ ]:
mount.park()

After you have verified the park position, you can unpark again and then go back to the home position. 

In [ ]:
mount.unpark()
mount.slew_to_home()

# Camera testing commands

The cameras are stored in `pocs.observatory.cameras` and are each referenced by a name like `Cam00` and `Cam01`.

Here we loop over each camera and use the `take_exposure` method, saving a unique filename for each.

In [ ]:
# For a unique filename.
now = current_time(flatten=True)

for cam_name, cam in pocs.observatory.cameras.items():
    cam.take_exposure(seconds=2, filename=f'/home/panoptes/images/{cam_name}-test-{now}.cr2', blocking=True)

# Scheduler testing commands

To schedule an observation the `pocs.observatory` will look at all the potential targets in the fields file and try to determine the "best" observation. 

In [ ]:
pocs.observatory.get_observation()

It's possible that there will not be any valid observations available.

If a valid observation is found, it will be assigned to `pocs.observatory.current_observation` and also tells the mount about the target. You can then do things like:

In [ ]:
# Make sure the mount is unparked
pocs.observatory.mount.unpark()

# Move to the target.
pocs.observatory.mount.slew_to_target()